<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/Copy_of_Semi_blind_MLP_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf
import random
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# Generating datasets and preprocessing

In [ ]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j=complex(0,1)
def fading_channel (Nt,Nr):
  h_real=np.random.normal(0, 0.5, size=(Nr, Nt))
  h_img=np.random.normal(0, 0.5, size=(Nr, Nt))
  H=np.array(h_real + h_img*j, dtype=complex)
  return H

In [ ]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4,4))
Constellation_set[0]= np.array([1,0,0,0])
Constellation_set[1]= np.array([0,1,0,0])
Constellation_set[2]= np.array([0,0,1,0])
Constellation_set[3]= np.array([0,0,0,1])

In [ ]:
H=fading_channel (4,2)

In [ ]:
'''Complex noise'''
def noise (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  n_img=np.random.normal(0, N0, size=Nr)
  noise=np.array(n_real + n_img*j, dtype=complex)
  return noise

In [ ]:
'''Creating pilot signals'''
def training (Label):
  i= 0
  y= np.zeros(shape= (4,2),dtype = "complex_")
  for i in range(Label):
    y[i]=np.dot(H, Constellation_set[i], out=None)+noise (0.3,2)
    i=i+1
  y_real=y.real
  y_imag=y.imag
  y1= np.concatenate((y_real,y_imag), axis=1)
  y2= pd.DataFrame(y1, columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
  y2['Label']=np.arange(0,4)
  return y2

In [ ]:
training (4)

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.911633,1.487851,-0.352323,-0.290797,0
1,0.922463,0.399867,-0.495622,-0.129701,1
2,-0.215056,0.442316,0.361124,0.288289,2
3,0.794832,-0.142618,0.309549,0.010833,3


In [ ]:
'''Creating test signals'''
def test_ (size):
  y3=pd.DataFrame()
  for i in range(size):
    y= np.zeros(shape=(1,2),dtype = "complex_")
    rand=random.randrange(0, 4, 1)
    y=np.dot(H, Constellation_set[rand], out=None)+noise (0.3,2)
    y_real=y.real
    y_imag=y.imag
    y1= np.concatenate((y_real,y_imag), axis=None)
    y2=pd.DataFrame(y1.reshape((1,4)),columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
    if rand == 0 :
      y2['Label']= 0
    elif rand == 1 :
      y2['Label']= 1
    elif rand == 2 :
      y2['Label']= 2
    elif rand == 3 :
      y2['Label']= 3
    y3=pd.concat([y3,y2])
  return y3

In [ ]:
test=test_ (2000)

In [ ]:
test

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.156717,1.021314,0.430401,0.316594,2
0,-0.134858,0.216980,0.157523,0.190325,2
0,0.818344,1.500191,-0.011949,-0.694231,0
0,0.037649,0.467689,-0.988157,-0.153773,1
0,0.120657,0.769165,-0.770513,0.549457,1
...,...,...,...,...,...
0,-0.273758,-0.134281,0.232637,-0.033292,2
0,-0.137803,-0.236525,0.473641,-0.070717,3
0,0.700472,-0.473087,0.207076,-0.172992,3
0,-0.661386,0.288964,0.495931,0.329054,2


In [ ]:
m1=test[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
n1=test['Label']
m1_NN=test[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
n1_NN=test['Label']
n1_NN=to_categorical(n1, num_classes=4)

# Generating virtual pilots (data augmentation)

In [ ]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  return n_real

In [ ]:
'''Data augmentation'''
q1=training (4)
l=pd.DataFrame()
for i in range(500):
  qs=q1[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]+noise_real(0.08,4)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1

In [ ]:
'''Data augmentation
q2=training (4)
i=0
for i in range(10):
  qs=q2[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq2=training (4)\ni=0\nfor i in range(10):\n  qs=q2[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

In [ ]:
'''Data augmentation
q3=training (4)
i=0
for i in range(10):
  qs=q3[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq3=training (4)\ni=0\nfor i in range(10):\n  qs=q3[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

In [ ]:
'''Data augmentation
q4=training (4)
i=0
for i in range(10):
  qs=q4[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.03,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq4=training (4)\ni=0\nfor i in range(10):\n  qs=q4[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.03,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

# Split dataset to train and test

In [ ]:
'''Spliting features and labels'''
#xy=pd.concat([l,q1])       #Enable when you want vp for SVM & KNN
xy=q1                       #Enable when you dont want vp for SVM & KNN
xy_NN=pd.concat([l,q1])     #Enable when you want vp for NN
#xy_NN=q1                   #Enable when you dont want vp for NN

In [ ]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,1.055780,1.361979,-0.038268,-0.337070,0
1,0.875746,0.939822,-0.759560,0.041247,1
2,0.126277,0.569460,0.519777,0.750072,2
3,0.551574,0.256281,0.124663,-0.049464,3


In [ ]:
xy_NN

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,1.097513,1.330416,0.049237,-0.449648,0
1,0.917479,0.908258,-0.672055,-0.071331,1
2,0.168010,0.537896,0.607282,0.637494,2
3,0.593307,0.224717,0.212168,-0.162043,3
0,1.039448,1.505742,-0.045933,-0.351884,0
...,...,...,...,...,...
3,0.536157,0.171276,0.095068,-0.085200,3
0,1.055780,1.361979,-0.038268,-0.337070,0
1,0.875746,0.939822,-0.759560,0.041247,1
2,0.126277,0.569460,0.519777,0.750072,2


In [ ]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.01, random_state=101)

In [ ]:
'''Spliting features and labels for SVM & KNN'''
x=xy[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y=xy['Label']

In [ ]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,1.055780,1.361979,-0.038268,-0.337070,0
1,0.875746,0.939822,-0.759560,0.041247,1
2,0.126277,0.569460,0.519777,0.750072,2
3,0.551574,0.256281,0.124663,-0.049464,3


In [ ]:
'''Spliting data to train and validation'''
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.000001, random_state=101)
X_train=x
y_train=y

In [ ]:
X_train.shape

(4, 4)

In [ ]:
y_train.shape

(4,)

# Building neural network and training

In [ ]:
'''Creating neural network'''
model = Sequential()
'''Creating neural network'''
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 4)                 132       
                                                                 
Total params: 8,644
Trainable params: 8,644
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
NN=model.fit(X_train_NN, y_train_NN, batch_size=16, epochs=2, validation_data=(X_test_NN,y_test_NN))

Epoch 1/2
124/124 [==============================] - 3s 9ms/step - loss: 0.5619 - accuracy: 0.8185 - val_loss: 0.0257 - val_accuracy: 1.0000
Epoch 2/2
124/124 [==============================] - 1s 5ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 1.0000


# Training KNN and SVM model

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

# Testing models on test dataset 

In [ ]:
poly_pred = poly.predict(m1)
rbf_pred = rbf.predict(m1)
knn_pred = knn.predict(m1)

In [ ]:
m1

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag
0,0.156717,1.021314,0.430401,0.316594
0,-0.134858,0.216980,0.157523,0.190325
0,0.818344,1.500191,-0.011949,-0.694231
0,0.037649,0.467689,-0.988157,-0.153773
0,0.120657,0.769165,-0.770513,0.549457
...,...,...,...,...
0,-0.273758,-0.134281,0.232637,-0.033292
0,-0.137803,-0.236525,0.473641,-0.070717
0,0.700472,-0.473087,0.207076,-0.172992
0,-0.661386,0.288964,0.495931,0.329054


In [ ]:
knn_accuracy = accuracy_score(n1, knn_pred)
knn_f1 = f1_score(n1, knn_pred, average='weighted')
print('Accuracy (KNN): ', "%.2f" % (knn_accuracy*100))
print('F1 (KNN): ', "%.2f" % (knn_f1*100))

Accuracy (KNN):  83.55
F1 (KNN):  84.12


In [ ]:
poly_accuracy = accuracy_score(n1, poly_pred)
poly_f1 = f1_score(n1, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  58.40
F1 (Polynomial Kernel):  57.69


In [ ]:
rbf_accuracy = accuracy_score(n1, rbf_pred)
rbf_f1 = f1_score(n1, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  83.55
F1 (RBF Kernel):  84.12


In [ ]:
'''Test the model'''
NN_Accuracy=model.evaluate(m1_NN, n1_NN)

63/63 [==============================] - 0s 4ms/step - loss: 0.5454 - accuracy: 0.8380


#Choosing K value

In [ ]:
'''error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(m1)
    error_rate.append(np.mean(pred_i != n1))'''

'error_rate = []\n\nfor i in range(1,40):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(X_train,y_train)\n    pred_i = knn.predict(m1)\n    error_rate.append(np.mean(pred_i != n1))'

In [ ]:
'''plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)
plt.title('Error Rate vs. K Values')
plt.xlabel('K')
plt.ylabel('Error Rate')'''

"plt.figure(figsize=(10,6))\nplt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)\nplt.title('Error Rate vs. K Values')\nplt.xlabel('K')\nplt.ylabel('Error Rate')"